In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  2.0.0-rc0
Eager mode:  True
Hub version:  0.6.0
GPU is available


In [3]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

# /Users/jheaton/tensorflow_datasets/imdb_reviews/plain_text/0.1.0

    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


In [0]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)

In [5]:
train_examples[:3]

array([b'I have no idea what the other reviewer is talking about- this was a wonderful movie, and created a sense of the era that feels like time travel. The characters are truly young, Mary is a strong match for Byron, Claire is juvenile and a tad annoying, Polidori is a convincing beaten-down sycophant... all are beautiful, curious, and decadent... not the frightening wrecks they are in Gothic.<br /><br />Gothic works as an independent piece of shock film, and I loved it for different reasons, but this works like a Merchant and Ivory film, and was from my readings the best capture of what the summer must have felt like. Romantic, yes, but completely rekindles my interest in the lives of Shelley and Byron every time I think about the film. One of my all-time favorites.',
       b'This was a wonderfully clever and entertaining movie that I shall never tire of watching many, many times. The casting was magnificent in matching up the young with the older characters. There are those of us

%% **Loading the pretrained Google "gnews" model which will convert the raw data into 20 dimensional vector **

In [0]:

model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)

In [10]:
# Raw training data of the movie review  
train_examples[:3]

array([b'I have no idea what the other reviewer is talking about- this was a wonderful movie, and created a sense of the era that feels like time travel. The characters are truly young, Mary is a strong match for Byron, Claire is juvenile and a tad annoying, Polidori is a convincing beaten-down sycophant... all are beautiful, curious, and decadent... not the frightening wrecks they are in Gothic.<br /><br />Gothic works as an independent piece of shock film, and I loved it for different reasons, but this works like a Merchant and Ivory film, and was from my readings the best capture of what the summer must have felt like. Romantic, yes, but completely rekindles my interest in the lives of Shelley and Byron every time I think about the film. One of my all-time favorites.',
       b'This was a wonderfully clever and entertaining movie that I shall never tire of watching many, many times. The casting was magnificent in matching up the young with the older characters. There are those of us

In [11]:
# Converting the raw text files into 20 dimensional vetors
hub_layer(train_examples[:3])

<tf.Tensor: id=496, shape=(3, 20), dtype=float32, numpy=
array([[ 3.0298512 , -3.1155329 ,  3.231569  , -2.470162  , -3.6778672 ,
        -2.7249148 , -1.5884265 ,  0.8121041 ,  4.3448696 , -1.4254425 ,
        -1.9567002 ,  0.7446213 , -0.9539236 ,  0.4037092 , -4.982661  ,
         0.9707939 ,  3.717819  , -1.3615017 , -2.881222  , -1.2961531 ],
       [ 3.9022908 , -5.336768  ,  4.6433706 , -2.322609  , -5.6409845 ,
        -2.2914128 , -1.3552328 ,  1.1260422 ,  4.928207  , -1.4990277 ,
        -3.5764084 ,  0.8652999 , -1.8267108 ,  0.7212572 , -6.1661105 ,
        -0.67619115,  5.83987   , -2.078528  , -3.7790735 , -2.0113754 ],
       [ 2.2488396 , -1.4537774 ,  1.97866   , -0.73290443, -2.2218246 ,
        -4.054772  , -1.6585274 ,  1.8872426 ,  1.8315402 ,  0.45302168,
        -0.50105846,  1.3690453 , -1.9862492 ,  0.4043505 , -5.337918  ,
         1.5515825 ,  3.6844683 , -3.3751655 , -3.4578393 , -1.1764543 ]],
      dtype=float32)>

In [13]:
# Build the network with the hub layer and adding 16 hidden layer and one output layer since binary classification
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compile network 
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
# Split the data into 10000 samples for validation and remaining 15000 samples for train set
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [16]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


15000/15000 [==============================] - 4s 235us/sample - loss: 0.7878 - accuracy: 0.5608 - val_loss: 0.6961 - val_accuracy: 0.6032
Epoch 2/40
15000/15000 [==============================] - 3s 168us/sample - loss: 0.6662 - accuracy: 0.6219 - val_loss: 0.6170 - val_accuracy: 0.6691
Epoch 3/40
15000/15000 [==============================] - 3s 171us/sample - loss: 0.5905 - accuracy: 0.6941 - val_loss: 0.5692 - val_accuracy: 0.7139
Epoch 4/40
15000/15000 [==============================] - 3s 177us/sample - loss: 0.5447 - accuracy: 0.7415 - val_loss: 0.5339 - val_accuracy: 0.7498
Epoch 5/40
15000/15000 [==============================] - 3s 180us/sample - loss: 0.5055 - accuracy: 0.7732 - val_loss: 0.5034 - val_accuracy: 0.7712
Epoch 6/40
15000/15000 [==============================] - 3s 176us/sample - loss: 0.4698 - accuracy: 0.7972 - val_loss: 0.4746 - val_accuracy: 0.7877
Epoch 7/40
15000/15000 [==============================] - 3s 174us/sample - loss: 0.4353 - accuracy: 0.8165 - v

In [18]:
# Evaluate the model 
results = model.evaluate(test_data, test_labels, verbose=0)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.402
accuracy: 0.857
